In [ ]:
from pathlib import Path

import ipywidgets as widgets

# Avoid non-compliant Type 3 fonts
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42  # pylint: disable=wrong-import-position

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact
from IPython.display import display
from tqdm import tqdm_notebook as tqdm

import utils

In [ ]:
pd.set_option('display.max_rows', None)
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
logs_dir = Path('logs')
eval_dir = logs_dir.parent / 'eval'
obstacle_configs = ['small_empty', 'small_columns', 'large_columns', 'large_divider']

In [ ]:
def load_data(cfg):
    eval_path = eval_dir / '{}.npy'.format(cfg.run_name)
    if eval_path.exists():
        return np.load(eval_path, allow_pickle=True)
    #print('Eval file for {} was not found'.format(cfg.run_name))
    return None

In [ ]:
def show_table():
    all_data = {}
    for log_dir in tqdm(list(logs_dir.iterdir())):
        # Load eval data for run
        cfg = utils.read_config(str(log_dir / 'config.yml'))
        data = load_data(cfg)
        if data is None:
            continue

        # Add mean cubes for run
        if cfg.experiment_name not in all_data:
            all_data[cfg.experiment_name] = []
        mean_cubes = np.mean([episode[-1]['cubes'] for episode in data])
        all_data[cfg.experiment_name].append(mean_cubes)

    # Replace runs with mean/std of runs
    for experiment_name, cubes_list in all_data.items():
        all_data[experiment_name] = '{:.2f} ± {:.2f}'.format(np.mean(cubes_list), np.std(cubes_list))

    # Display in table
    def f(obstacle_config):
        data = {'cubes': {
            experiment_name: cubes for experiment_name, cubes in all_data.items()
            if experiment_name.startswith(obstacle_config)
        }}
        display(pd.DataFrame(data))
    interact(f, obstacle_config=widgets.Dropdown(options=obstacle_configs))

In [ ]:
show_table()

In [ ]:
def extend_curves(curves):
    if len(curves) == 0:
        return curves
    max_length = max(len(curve) for curve in curves)
    for i, curve in enumerate(curves):
        curves[i] = np.pad(curve, (0, max_length - len(curve)), 'edge')
    return curves

In [ ]:
def get_curve_for_run(data, step_size):
    curves = []
    for episode in data:
        cubes = np.asarray([step['cubes'] for step in episode])
        cumulative_distance = np.asarray([step['distance'] for step in episode])
        x = np.arange(0, cumulative_distance[-1] + step_size, step_size)
        xp, fp = cumulative_distance, cubes
        curves.append(np.floor(np.interp(x, xp, fp, left=0)))
    curves = extend_curves(curves)
    return np.mean(curves, axis=0)

In [ ]:
def get_label(experiment_name):
    parts = experiment_name.split('-')
    if len(parts) == 1:
        return 'Ours'
    return {
        'fixed_step_size': 'Ours, fixed step size',
        'steering_commands': 'Steering commands',
        'no_partial_rewards': 'Ours, no partial rewards',
        'no_sp_components': 'Ours, no shortest path components',
        'no_sp_from_agent': 'Ours, no sp from agent',
        'no_sp_to_receptacle': 'Ours, no sp to receptacle',
        'no_sp_movement': 'Ours, no sp movement',
        'no_sp_in_rewards': 'Ours, no sp in rewards',
        'steering_commands-no_sp_components': 'Steering commands, no sp',
    }['-'.join(parts[1:])]

In [ ]:
def show_curves():
    step_size = 0.1

    all_curves = {}
    for log_dir in tqdm(list(logs_dir.iterdir())):
        # Load eval data for run
        cfg = utils.read_config(str(log_dir / 'config.yml'))
        data = load_data(cfg)
        if data is None:
            continue

        # Add curve for run
        if cfg.experiment_name not in all_curves:
            all_curves[cfg.experiment_name] = []
        all_curves[cfg.experiment_name].append(get_curve_for_run(data, step_size))

    def plot_curves(obstacle_config, experiment_names, fontsize=20):
        for experiment_name in experiment_names:
            curves = extend_curves(all_curves[experiment_name])
            x = np.arange(0, (len(curves[0]) - 0.5) * step_size, step_size)
            y_mean = np.mean(curves, axis=0)
            y_std = np.std(curves, axis=0)
            plt.plot(x, y_mean, label=get_label(experiment_name))
            plt.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.2)
        plt.xlabel('Distance (m)', fontsize=fontsize)
        plt.ylabel('Num Blocks', fontsize=fontsize)
        if obstacle_config == 'large_divider':
            plt.xlim(0, 120)
        num_cubes = 20 if obstacle_config.startswith('large') else 10
        plt.ylim(0, num_cubes)
        plt.xticks(fontsize=fontsize - 2)
        plt.yticks(range(0, num_cubes + 1, 2), fontsize=fontsize - 2)
        plt.legend(fontsize=fontsize - 2, loc='upper left')

    def f(obstacle_config, experiment_names, save_to_pdf):
        if len(experiment_names) == 0:
            return
        plot_curves(obstacle_config, experiment_names)
        if save_to_pdf:
            plt.savefig('curves-{}.pdf'.format(obstacle_config), bbox_inches='tight')
        else:
            plt.show()

    obstacle_config_dropdown = widgets.Dropdown(options=obstacle_configs)
    experiment_names_select = widgets.SelectMultiple(layout=widgets.Layout(width='50%'))
    save_toggle = widgets.ToggleButton(description='Save to PDF')
    def update_experiment_names_options(*_):
        matching_experiment_names = []
        for experiment_name in all_curves:
            if experiment_name.startswith(obstacle_config_dropdown.value):
                matching_experiment_names.append(experiment_name)
        experiment_names_select.options = matching_experiment_names
        experiment_names_select.rows = len(matching_experiment_names)
    obstacle_config_dropdown.observe(update_experiment_names_options)

    interact(f, obstacle_config=obstacle_config_dropdown,
             experiment_names=experiment_names_select, save_to_pdf=save_toggle)

In [ ]:
show_curves()